<a href="https://colab.research.google.com/github/Bread806/BreadTestPage/blob/main/goldbach_0223_MLP%2Bn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [3]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
import math
from sklearn.preprocessing import StandardScaler


def is_prime(num):
    """檢查一個數字是否為質數"""
    if num <= 1:
        return False
    for i in range(2, int(num ** 0.5) + 1):
        if num % i == 0:
            return False
    return True


def prime_table(x):
    """建立小於x的質數表"""
    primes = [num for num in range(2, x) if is_prime(num)]
    return primes


def convert_base_into_list(number, base, width=10):
    result = []  # init list

    for i in range(width):
        result.append(number % base)
        number = number // base
    result = result[::-1]
    return result


def prime_to_index(primeSize, primes, number):
    for index in range(primeSize):
        if number == primes[index]:
            return index
    return -1

##loading data

In [ ]:
# shuffle

print("---loading data---")
# load data
trainData = pd.read_csv('/content/Drive/MyDrive/實驗/goldbach/csv/traindata_DividebyG4.csv')
trainDataNumbers = trainData['Number'].values
trainDataLabel = trainData['Factors'].values
print("---loading data done.---")

# shuffle training data
shuffleIndices = np.random.permutation(len(trainDataNumbers))
shuffledNumbers = trainDataNumbers[shuffleIndices]
shuffledLabel = trainDataLabel[shuffleIndices]

---loading data---
---loading data done.---


##parse feature

In [ ]:
mergedTrainNumber = []
mergedTrainLabel = []

for num in shuffledNumbers:
    feature = []
    # base(2,3,5,7) and itself

    itself = [num / 2000000]
    L = [math.log(num)]
    feature = convert_base_into_list(num, 2)+convert_base_into_list(num, 3)+convert_base_into_list(num, 5)+convert_base_into_list(num, 7) + itself + L
    mergedTrainNumber.append(feature)

for num in shuffledLabel:
    mergedTrainLabel.append(num)

##spit train set and val set

In [ ]:
mergedArray = np.array(mergedTrainNumber)
mergedLabel = np.array(mergedTrainLabel)

X_train, X_val, y_train, y_val = train_test_split(mergedArray, mergedLabel, test_size=0.2, random_state=42)
#print (X_train[0])

##start training

In [ ]:
# start training
# 清空所有
keras.clear_session()

################################################################

# require gpu
device = "/device:GPU:0"
with tf.device(device):
    print(f"--- gpu {device} is loaded. ---")
    model = Sequential()

    model.add(Dense(units=42, input_shape=(42,), activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=1, activation='linear'))

    print(model.summary())

    # model.compile(loss='binary_crossentropy', optimizer='adam')   # train version 1
    custom_adam_optimizer = Adam(learning_rate=0.001)
    model.compile(loss='MSE', optimizer=custom_adam_optimizer,
                  metrics=['accuracy'])    # train version 2

    # 训练模型
    print("---starting training---")
    trainHistory = model.fit(X_train, y_train, epochs=1000,batch_size=2048, validation_data=(X_val, y_val))
    print("--------------------------------------------------------")

    # save model
    model.save('goldbach_model_1020_400w.h5')
print("model saved.")

# predict

In [1]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [4]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [5]:
## load data
#predict

from tensorflow.keras.models import load_model
import csv
import pandas as pd
import numpy as np

# 指定 HDF5 模型文件的路徑
modelPath = '/content/Drive/MyDrive/實驗/model_save/goldbach_model_0222_MLPn.h5'

# 使用 load_model
model = load_model(modelPath)

# load test_set and prime data
testDataPath = '/content/Drive/MyDrive/實驗/goldbach/csv/test_set_G4_7M.csv'
testData = pd.read_csv(testDataPath)
testDataNumbers = testData['Number'].values
testDataLabel = testData['Factors'].values
testDataG4 = testData['G4'].values
testDataPartition = testData['Partition'].values

print("---loading test data done.---")


---loading test data done.---


In [6]:
# shuffle training data
shuffleIndices = np.random.permutation(len(testDataNumbers))
shuffledNumbers = testDataNumbers[shuffleIndices]
shuffledLabel = testDataLabel[shuffleIndices]

In [7]:
# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [8]:
# collect data
# prepare training set
mergedTestNumber = []
mergedTestLabel = []

for num in shuffledNumbers:

    feature = []
    # base(2,3,5,7) and itself

    itself = [num / 2000000]
    L = [math.log(num)]
    feature = convert_base_into_list(num, 2)+convert_base_into_list(num, 3)+convert_base_into_list(num, 5)+convert_base_into_list(num, 7) + itself + L
    mergedTestNumber.append(feature)

for num in shuffledLabel:
    mergedTestLabel.append(num)

    #print("merge number ", num, " is done.")

#print (mergedTrainNumber)

In [9]:
mergedArray = np.array(mergedTestNumber)
mergedLabel = np.array(mergedTestLabel)

In [10]:
evaluation = model.evaluate(mergedArray, mergedLabel)
XNewData = testDataNumbers, testDataLabel
print ("Loss:", evaluation[0])
print ("Accuracy:", evaluation[1])
print ("##########################")
predictions = model.predict(mergedArray)

2/2 [==============================] - 0s 10ms/step - loss: 0.0202 - accuracy: 0.0000e+00
Loss: 0.02019587904214859
Accuracy: 0.0
##########################
2/2 [==============================] - 0s 6ms/step


In [12]:
"""
訓練是練partition跟G4的關係，所以這邊要對label 正規化 G4
"""
print (f"將50筆測資正規化回G4")
normalizedTest = []
normalizedPred = []
for index in range(len(testDataLabel)):
  normalizedTest.append(testDataLabel[index]*testDataG4[index]) # normalize ground truth
  normalizedPred.append(predictions[index]*testDataG4[index])  # normalize predition

將50筆測資正規化回G4


In [13]:
print (f"將50筆測資正規化回G4")
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]:05d}\t模型預估 = {int(normalizedPred[index])}")

將50筆測資正規化回G4
第0筆資料為數字:	7984914	|正確答案 = 58853	模型預估 = 24663
第1筆資料為數字:	7050792	|正確答案 = 51667	模型預估 = 32571
第2筆資料為數字:	7852768	|正確答案 = 31367	模型預估 = 32414
第3筆資料為數字:	7807448	|正確答案 = 26161	模型預估 = 29141
第4筆資料為數字:	7695230	|正確答案 = 31001	模型預估 = 47832
第5筆資料為數字:	7109000	|正確答案 = 28859	模型預估 = 23263
第6筆資料為數字:	7268430	|正確答案 = 64326	模型預估 = 50732
第7筆資料為數字:	7147384	|正確答案 = 22306	模型預估 = 29873
第8筆資料為數字:	7555772	|正確答案 = 27371	模型預估 = 46147
第9筆資料為數字:	7066874	|正確答案 = 21654	模型預估 = 22209
第10筆資料為數字:	7632520	|正確答案 = 36879	模型預估 = 31623
第11筆資料為數字:	7039812	|正確答案 = 46596	模型預估 = 44276
第12筆資料為數字:	7670862	|正確答案 = 46176	模型預估 = 31842
第13筆資料為數字:	7402232	|正確答案 = 22333	模型預估 = 28581
第14筆資料為數字:	7432508	|正確答案 = 22866	模型預估 = 55735
第15筆資料為數字:	7230862	|正確答案 = 22140	模型預估 = 54061
第16筆資料為數字:	7305346	|正確答案 = 22305	模型預估 = 60313
第17筆資料為數字:	7349124	|正確答案 = 47254	模型預估 = 67848
第18筆資料為數字:	7134356	|正確答案 = 22965	模型預估 = 26811
第19筆資料為數字:	7990454	|正確答案 = 23942	模型預估 = 25609
第20筆資料為數字:	7024954	|正確答案 = 22002	模型預估 = 46886
第21筆資料為數字:	7715734	|正確答案 = 2551

<ipython-input-13-0f6ad44a8f98>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]:05d}\t模型預估 = {int(normalizedPred[index])}")


In [14]:
def calculate_mse(predictions, targets):
    """
    計算均方誤差 (MSE)

    參數:
    predictions: 預測值的列表或陣列
    targets: 實際值的列表或陣列，與預測值對應

    返回值:
    mse: 均方誤差
    """
    # 確保預測值和實際值的長度相等
    if len(predictions) != len(targets):
        raise ValueError("預測值和實際值的長度不一致")

    # 計算平方誤差
    squared_errors = [(p - t) ** 2 for p, t in zip(predictions, targets)]

    # 計算均方誤差
    mse = sum(squared_errors) / len(predictions)

    return mse


mean = testData['Partition'].mean()

#LN MSE
MSE = int(calculate_mse(normalizedPred, testDataPartition))

#MLP MSE
RMSE= math.sqrt(MSE)
errorRate = RMSE/mean

print (f"MSE : {MSE}\nRMSE : {RMSE}\nError Rate : {errorRate}  ->  {round(errorRate,7)*100}%")

MSE : 429059712
RMSE : 20713.756588315893
Error Rate : 0.5883219417157337  ->  58.83219%


<ipython-input-14-7a04c2639256>:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  MSE = int(calculate_mse(normalizedPred, testDataPartition))
